In [2]:
pip install --upgrade selenium

  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.8.3-py3-none-any.whl.metadata (2.4 kB)
   ---------------------------------------- 0.0/9.6 MB ? eta -:--:--
   ----- ---------------------------------- 1.3/9.6 MB 7.5 MB/s eta 0:00:02
   ---------- ----------------------------- 2.6/9.6 MB 6.3 MB/s eta 0:00:02
   ---------------- ----------------------- 3.9/9.6 MB 6.5 MB/s eta 0:00:01
   --------------------- ------------------ 5.2/9.6 MB 6.6 MB/s eta 0:00:01
   -------------------------- ------------- 6.3/9.6 MB 6.0 MB/s eta 0:00:01
   ------------------------------ --------- 7.3/9.6 MB 6.0 MB/s eta 0:00:01
   ---------------------------------- ----- 8.4/9.6 MB 5.8 MB/s eta 0:00:01
   ---------------------------------------  9.4/9.6 MB 5.8 MB/s eta 0:00:01
   ---------------------------------------- 9.6/9.6 MB 5.5 MB/s eta 0:00:00
Using cached certifi-2025.8.3-py3-none-any.whl (161 kB)
Using cached urllib3-2.5.0-py3-none-any.whl (129 kB)
  Attem

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.37.1 requires protobuf<6,>=3.20, but you have protobuf 6.32.1 which is incompatible.


In [ ]:
# zigzag_local_crawler.py (Selenium Manager 최종 버전)

import os
import time
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

# --- 설정 ---
KEYWORDS = ["모던", "페미닌", "캐주얼", "스트릿"] 
IMAGES_TO_CRAWL = 100
# --- 설정 끝 ---

def setup_driver():
    """Selenium 4.6+의 Selenium Manager를 사용하여 자동으로 드라이버를 설정합니다."""
    # 특별한 옵션 없이 Chrome을 실행하면 Selenium Manager가 알아서 처리합니다.
    driver = webdriver.Chrome()
    driver.implicitly_wait(10)
    return driver

def scroll_to_load_images(driver, target_count):
    image_urls = set()
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    wait = WebDriverWait(driver, 20)
    try:
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.zds4_1hj32t61")))
        print("이미지 목록 로딩을 확인했습니다.")
    except TimeoutException:
        print("시간 초과: 페이지에서 이미지 목록을 찾을 수 없습니다.")
        driver.save_screenshot('timeout_screenshot.png')
        print("현재 페이지의 스크린샷을 'timeout_screenshot.png'로 저장했습니다.")
        return []

    while len(image_urls) < target_count:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        
        # --- 핵심 수정: 이미지를 포함하는 div의 클래스를 이용해 img 태그를 찾습니다 ---
        image_containers = driver.find_elements(By.CSS_SELECTOR, "div.zds4_1hj32t61")
        
        for container in image_containers:
            try:
                # 각 컨테이너 안의 img 태그를 찾습니다.
                img_element = container.find_element(By.TAG_NAME, "img")
                src = img_element.get_attribute('src')
                if src and src.startswith('https'):
                    image_urls.add(src)
            except:
                # div 안에 img 태그가 없는 경우를 대비해 예외 처리
                continue
                
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            print("더 이상 로드할 이미지가 없습니다."); break
        last_height = new_height
        print(f"현재 수집된 이미지 URL 개수: {len(image_urls)}개")
        
    return list(image_urls)

def download_images(keyword, urls, max_count):
    save_dir = f"./{keyword}"
    os.makedirs(save_dir, exist_ok=True)
    print(f"\n'{keyword}' 키워드 이미지 다운로드를 시작합니다. (총 {min(len(urls), max_count)}개)")
    for i, url in enumerate(urls):
        if i >= max_count: break
        try:
            response = requests.get(url, stream=True, timeout=10)
            if response.status_code == 200:
                filename = f"{str(i+1).zfill(3)}.jpg"
                filepath = os.path.join(save_dir, filename)
                with open(filepath, 'wb') as f: f.write(response.content)
                print(f"({i+1}/{max_count}) {filename} 다운로드 완료")
        except Exception as e: print(f"이미지 다운로드 중 오류 발생: {url} ({e})")

def crawl_zigzag():
    driver = None
    try:
        driver = setup_driver()
        for keyword in KEYWORDS:
            print(f"\n--- '{keyword}' 키워드 크롤링 시작 ---")
            search_url = f"https://zigzag.kr/search?keyword={keyword}&order=SCORE_DESC"
            print(f"접속할 URL: {search_url}")
            driver.get(search_url)
            
            image_urls = scroll_to_load_images(driver, IMAGES_TO_CRAWL)
            if image_urls:
                download_images(keyword, image_urls, IMAGES_TO_CRAWL)

    except Exception as e:
        print(f"\n크롤링 중 오류가 발생했습니다:")
        import traceback
        traceback.print_exc()
    finally:
        if driver:
            driver.quit()
        print("\n--- 모든 크롤링 작업이 완료되었습니다. ---")

# --- 메인 코드 실행 ---
if __name__ == '__main__':
    crawl_zigzag()


--- '모던' 키워드 크롤링 시작 ---
접속할 URL: https://zigzag.kr/search?keyword=모던&order=SCORE_DESC
이미지 목록 로딩을 확인했습니다.
현재 수집된 이미지 URL 개수: 9개
현재 수집된 이미지 URL 개수: 18개
현재 수집된 이미지 URL 개수: 27개
현재 수집된 이미지 URL 개수: 36개
현재 수집된 이미지 URL 개수: 45개
현재 수집된 이미지 URL 개수: 54개
현재 수집된 이미지 URL 개수: 63개
현재 수집된 이미지 URL 개수: 72개
현재 수집된 이미지 URL 개수: 81개
현재 수집된 이미지 URL 개수: 90개
현재 수집된 이미지 URL 개수: 99개
현재 수집된 이미지 URL 개수: 108개

'모던' 키워드 이미지 다운로드를 시작합니다. (총 100개)
(1/100) 001.jpg 다운로드 완료
(2/100) 002.jpg 다운로드 완료
(3/100) 003.jpg 다운로드 완료
(4/100) 004.jpg 다운로드 완료
(5/100) 005.jpg 다운로드 완료
(6/100) 006.jpg 다운로드 완료
(7/100) 007.jpg 다운로드 완료
(8/100) 008.jpg 다운로드 완료
(9/100) 009.jpg 다운로드 완료
(10/100) 010.jpg 다운로드 완료
(11/100) 011.jpg 다운로드 완료
(12/100) 012.jpg 다운로드 완료
(13/100) 013.jpg 다운로드 완료
(14/100) 014.jpg 다운로드 완료
(15/100) 015.jpg 다운로드 완료
(16/100) 016.jpg 다운로드 완료
(17/100) 017.jpg 다운로드 완료
(18/100) 018.jpg 다운로드 완료
(19/100) 019.jpg 다운로드 완료
(20/100) 020.jpg 다운로드 완료
(21/100) 021.jpg 다운로드 완료
(22/100) 022.jpg 다운로드 완료
(23/100) 023.jpg 다운로드 완료
(24/100) 024.jpg